In [13]:
from google.colab import drive
import os

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
os.chdir('/content/gdrive/My Drive/High Dimension Analysis/Practice/Final/FaceMaskDataset')
print(os.getcwd())

/content/gdrive/My Drive/High Dimension Analysis/Practice/Final/FaceMaskDataset


In [15]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [16]:
def ShowImage(ImageList, nRows = 1, nCols = 2, WidthSpace = 0.00, HeightSpace = 0.00):    
    from matplotlib import pyplot as plt     
    import matplotlib.gridspec as gridspec        
    
    gs = gridspec.GridSpec(nRows, nCols)         
    gs.update(wspace=WidthSpace, hspace=HeightSpace) # set the spacing between axes.    
    plt.figure(figsize=(20,20))    
    for i in range(len(ImageList)):        
        ax1 = plt.subplot(gs[i])        
        ax1.set_xticklabels([])        
        ax1.set_yticklabels([])        
        ax1.set_aspect('equal') 
        
        plt.subplot(nRows, nCols,i+1) 
        image = ImageList[i].copy()        
        if (len(image.shape) < 3):            
            plt.imshow(image, plt.cm.gray)        
        else:            
            plt.imshow(image)        
            plt.title("Image " + str(i))        
            plt.axis('off') 
    plt.show()

In [17]:
def get_subfiles(dir):
  "Get a list of immediate subfiles"
  return next(os.walk(dir))[2]


## **Save image**

In [6]:
# def save_data(path, no_of_images, path_p): #function to make dataset
#     for i,j in enumerate(no_of_images):
#         if ((i%20 == 0) and (i != 0)):
#             print()
#         print(i, end = ' ')
#         tree = ElementTree.parse(path+j[:-4]+".xml")
#         root = tree.getroot()

#         if (root.find('object') == None):
#             continue

#         label_dt = []
#         for l in root.findall('.//name'):
#             if (l.text not in options):
#                 if ('_' in l.text):
#                     label = options['face_mask']
#                 else:
#                     label = options['face']
#             else:
#                 label=options[l.text]
#             label_dt.append(label)

#         t = 0
#         for box in root.findall('.//bndbox'):
#             a = int(box.find('xmin').text)
#             b = int(box.find('ymin').text)
#             c = int(box.find('xmax').text)+50
#             d = int(box.find('ymax').text)

#             image = cv2.imread(path+j)
#             # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             image = image[b:d, a:c]
#             area = image.shape[0]*image.shape[1]
#             image = cv2.resize(image, (224, 224))
#             if (area < 3000 and label_dt[t] == 0):
#                 del label_dt[t]
#                 # j -= 1
#                 continue
#             if (label_dt[t] == 0):
#                 cv2.imwrite(path_p+'/face/'+str(i)+'_'+str(t)+'.png', image)
#             else:
#                 cv2.imwrite(path_p+'/face_mask/'+str(i)+'_'+str(t)+'.png', image)
#             t += 1

In [7]:
# path_train = 'image_processing/train/'
# make_dataset(path_train)

In [8]:
# path_val = 'image_processing/val/'
# make_dataset(path_val)

## **Train model**

In [9]:
# USAGE
# python train_mask_detector.py --dataset dataset

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt

In [10]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
# imagePaths = '/content/gdrive/My Drive/shecode/reference2-paper/dataset/'

# os.path.exists(imagePaths)

[INFO] loading images...


In [11]:
def get_image_label(path):
		data = []
		labels = []
		for imagePath in os.listdir(path):
			# extract the class label from the filename
				label = imagePath
				i = 0
				print('\n',imagePath)
				for name in get_subfiles(path + imagePath):
						# load the input image (224x224) and preprocess it
						if (i == 6000):
								break
						if ((i%20 == 0) and (i != 0)):
								print()
						print(i, end = ' ')
						img = cv2.imread(path + imagePath + '/' + name)
						# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
						# img = cv2.resize(img, (224, 224))
						img_ = img_to_array(img)
						image = preprocess_input(img_)

						# update the data and labels lists, respectively
						data.append(image)
						labels.append(label)
						i += 1
		return data, labels

In [12]:
trainX, trainY = get_image_label('FaceMaskDataset/image_processing/train/')

# convert the data and labels to NumPy arrays
trainX = np.array(trainX, dtype="float32")
trainY = np.array(trainY)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
trainY = to_categorical(trainY)

FileNotFoundError: ignored

In [ ]:
valX, valY = get_image_label('FaceMaskDataset/image_processing/val/')

# convert the data and labels to NumPy arrays
valX = np.array(valX, dtype="float32")
valY = np.array(valY)

lb = LabelBinarizer()
valY = lb.fit_transform(valY)
valY = to_categorical(valY)

In [ ]:
# initialize the initial learning rate, number of epochs to train for, and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [ ]:
# TRAIN
# construct the training image generator for data augmentation
# load the MobileNetV2 network, ensuring the head FC layer sets are left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
		layer.trainable = False

# compile our model
print("[INFO] compiling model...")

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])

In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(trainX, trainY, 
              batch_size=BS,
              steps_per_epoch=len(trainX) // BS,
              validation_data=(valX, valY),
              validation_steps=len(valX) // BS,
              epochs=EPOCHS)

In [ ]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(valX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(valY.argmax(axis=1), 
                            predIdxs,
                            target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("model.h5")

# serialize model to JSON
# model_json = model.to_json()
# with open("./save_model/model.json", "w") as json_file:
#    json_file.write(model_json)
# serialize weights to HDF5
# model.save_weights("./save_model/model.h5")
print("Saved model to disk")
model.summary()


# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
# plt.savefig(args["plot"])

## **Load model**

In [22]:
# USAGE
# python detect_mask_image.py --image examples/example_01.png

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

print("[INFO] loading face mask detector model...")
model = load_model('model.h5')

[INFO] loading face mask detector model...


## **DETECT MASK IMAGE**

In [ ]:
testImg = []
pathPredict = "/content/gdrive/My Drive/High Dimension Analysis/Practice/Final/FaceMaskDataset/test/"
for name in get_subfiles(pathPredict):
	# load the input image (224x224) and preprocess it
	img = cv2.imread(pathPredict + name)
	img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
	testImg.append(img)

In [ ]:
ShowImage(testImg, 1, len(testImg))

In [ ]:
!pip install mtcnn #installing library for predicting faces

In [ ]:
from mtcnn import MTCNN
detect = MTCNN()

In [ ]:
def testing(image_orig):
    options = {0:"face_mask", 1:"face"} # mapping for predictions and analysis purpose
    image = image_orig.copy()
    detections = detect.detect_faces(image)
    for i in range(0, len(detections)):
      # extract the confidence (i.e., probability) associated with
      # the detection
      # confidence = detections[0, 0, i, 2]
      # filter out weak detections by ensuring the confidence is
      # greater than the minimum confidence
      if detections[i]['confidence'] > 0.9: #args["confidence"]:
        # compute the (x, y)-coordinates of the bounding box for
        # the object
        # box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x, y, width, height) = detections[i]['box']
        (x, y) = (max(0, x), max(0, y))
        # ensure the bounding boxes fall within the dimensions of the frame

        # extract the face ROI, convert it from BGR to RGB channel
        # ordering, resize it to 224x224, and preprocess it
        face = image[y:(y+height), x:(x+width)]

        # face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)

        # pass the face through the model to determine if the face
        # has a mask or not
        (mask, withoutMask) = model.predict(face)[0]

        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = 1 if mask > withoutMask else 0
        color = (0, 255, 0) if label == 1 else (255, 0, 255)

        # include the probability in the label
        label = "{}: {:.2f}%".format(options[label], max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(image, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (x, y), (x+width, y+height), color, 2)
    
    ShowImage([image_orig, image], 1, 2)

In [ ]:
testing(testImg[0])

In [ ]:
testing(testImg[1])

In [ ]:
testing(testImg[2])

In [ ]:
testing(testImg[3])

## **DETECT MASK VIDEO**

In [27]:
# USAGE
# python detect_mask_image.py --image examples/example_01.png

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

print("[INFO] loading face mask detector model...")
model = load_model('model.h5')

[INFO] loading face mask detector model...


In [29]:
!pip install mtcnn #installing library for predicting faces

     |████████████████████████████████| 2.3MB 8.6MB/s 


In [30]:
from mtcnn import MTCNN
detect = MTCNN()

In [ ]:
# USAGE
# python detect_mask_video.py

# import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def detect_and_predict_mask(frame):
    detections = detect.detect_faces(frame)
    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, len(detections)):
        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if detections[i]['confidence'] > 0.9:
        # compute the (x, y)-coordinates of the bounding box for the object
            (x, y, width, height) = detections[i]['box']
            (x, y) = (max(0, x), max(0, y))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[y:(y+height), x:(x+width)]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((x, y, x+width, y+height))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = model.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding
    # locations
    return (locs, preds)

In [ ]:
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)

while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=900)
    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame)
    options = {0:"face_mask", 1:"face"}
    
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = 1 if mask > withoutMask else 0
        color = (0, 255, 0) if label == 0 else (255, 0, 255)
        # include the probability in the label
        label = "{}: {:.2f}%".format(options[label], max(mask, withoutMask) * 100)
        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()